# Prompting Científico — Caso MalT (Protocolo Completo)

Este notebook implementa un flujo multi-etapa con control epistemológico distribuido.
Ejecuta cada fase secuencialmente y realiza revisión humana en los checkpoints indicados.


In [ ]:
%pip -q install --upgrade openai

In [ ]:
import os, json, datetime
from openai import OpenAI

client = OpenAI()

models = client.models.list()
available = [m.id for m in models.data]
MODEL = available[0]
MODEL

In [ ]:
def response_to_text(resp):
    chunks = []
    if hasattr(resp, 'output') and resp.output:
        for item in resp.output:
            if hasattr(item, 'content') and item.content:
                for c in item.content:
                    if getattr(c, 'type', None) == 'output_text':
                        chunks.append(getattr(c, 'text', ''))
    return '\n'.join(chunks).strip()

def call_llm(prompt, system=None):
    messages = []
    if system:
        messages.append({'role': 'system', 'content': system})
    messages.append({'role': 'user', 'content': prompt})
    resp = client.responses.create(model=MODEL, input=messages)
    return response_to_text(resp)

## Pegar aquí el texto del paper


In [ ]:
paper_text = '''
PEGA AQUI EL TEXTO DEL PAPER
'''
paper_text = paper_text.strip()
len(paper_text)

## Fase 1 — Exploración


In [ ]:
PROMPT_F1 = f'''
Rol: Explorador científico.
Identifica párrafos que mencionen MalT y evidencia regulatoria.
No interpretar. No resumir. Citar literalmente.

Texto:
{paper_text}
'''

f1_out = call_llm(PROMPT_F1)
print(f1_out[:2000])

## Fase 2 — Extracción literal (Checkpoint humano antes de ejecutar)


In [ ]:
PROMPT_F2 = f'''
Extrae únicamente información explícita sobre regulación mediada por MalT.
No inferir. Incluir cita textual.

Fragmentos:
{f1_out}
'''

f2_out = call_llm(PROMPT_F2)
print(f2_out[:2000])

## Fase 3 — Normalización


In [ ]:
PROMPT_F3 = f'''
Estandariza nombres génicos. Reporta ambigüedades.
Tabla de entrada:
{f2_out}
'''

f3_out = call_llm(PROMPT_F3)
print(f3_out[:2000])

## Fase 4 — Evaluación crítica (Revisión humana obligatoria)


In [ ]:
PROMPT_F4 = f'''
Clasifica evidencia como Directa, Indirecta o Correlacional.
Define criterios explícitos y justifica con cita textual.

Tabla:
{f3_out}
'''

f4_out = call_llm(PROMPT_F4)
print(f4_out[:2000])

## Fase 5 — Integración humana

El investigador integra, valida y decide.
